# Self Consistency Prompting

One of the more advanced techniques in prompt engineering is self-consistency, introduced by `Wang et al. (2022)`. 

This method seeks to improve upon the traditional greedy decoding typically used in chain-of-thought (CoT) prompting. 

The core concept involves sampling multiple diverse reasoning paths through few-shot CoT and leveraging these variations to determine the most consistent answer. The technique  enhances the effectiveness of CoT prompting, particularly for tasks requiring arithmetic and commonsense reasoning.

## References:
* [Wang et al. (2022)](https://arxiv.org/abs/2203.11171)

## Running this code on MyBind.org

Note: remember that you will need to **adjust CONFIG** with **proper URL and API_KEY**!

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/GenILab-FAU/prompt-eng/HEAD?urlpath=%2Fdoc%2Ftree%2Fprompt-eng%2Fself_consistency.ipynb)



In [ ]:
############################################################
## SELF-CONSISTENCY PROMPTING: PROJECT OVERVIEW
############################################################

import os
import csv
import random
from datetime import datetime
from _pipeline import create_payload, model_req

# 1) Read the last response from the chain-of-thought logs CSV file
with open("data/chain_of_thought_logs.csv", "r", encoding="utf-8") as f:
    reader = csv.reader(f)
    chain_of_thought_output = list(reader)[-1][5]  # Assuming the response is in the sixth column

# 2) Define a base prompt for multiple solutions
BASE_PROMPT = f"""
You are an advanced AI-driven technical writing system tasked with generating a **fully structured, high-quality, and professional technical report** in Markdown format. Your primary objective is to **refine and finalize a comprehensive project report** with a particular emphasis on the **'Results & Evaluation'** section. 

While the **main goal** is to ensure that the 'Results & Evaluation' section is the most insightful, technically sound, and well-structured, you must also maintain **a professional and logically coherent structure** across all preceding sections, including:
- **Project Overview** (Concise introduction to the project)
- **Data & Methodologies** (Thorough explanation of data collection, preprocessing, and analytical techniques)
- **Results & Evaluation** (Detailed insights, performance metrics, visualizations, and structured technical analysis)

The **final output** must be a **refined, high-quality, and structured Markdown-formatted report** that effectively conveys all essential information, ensuring seamless readability, technical precision, and a structured presentation.

---

## **Guidelines for Refinement**

### **1. Maintain Professional Quality Across All Sections**
- **Project Overview:** Ensure clarity, conciseness, and strong introduction of key objectives.
- **Data & Methodologies:** Present structured explanations of data sources, preprocessing, and analytical techniques.
- **Results & Evaluation:** Provide detailed insights, structured performance analysis, and well-integrated code blocks.

### **2. Generate & Compare Multiple Versions of 'Results & Evaluation'**
- Generate **three distinct versions** of the 'Results & Evaluation' section, each varying in structure, depth, and formatting.
- Each version must contain:
  - **Results Summary:** Main findings, trends, and anomalies.
  - **Evaluation Metrics:** Accuracy, precision, recall, F1-score, RMSE, etc.
  - **Data Visualizations:** Tables, figures, and structured charts.
  - **Code Blocks:** Performance analysis and metric computation.

### **3. Select and Refine the Best Version**
- Compare the versions using these criteria:
  - **Clarity & Readability:** Structured and easy to comprehend.
  - **Comprehensiveness:** Coverage of all relevant findings.
  - **Logical Flow:** Smooth transitions from results to evaluation.
  - **Technical Accuracy:** Correct calculations, visualizations, and justifications.
- Select the **best version** and refine it to meet the highest professional standards.

### **4. Apply Final Refinements to the Entire Report**
- Ensure the **Project Overview** and **Data & Methodologies** sections are logically structured and polished.
- Format the report with:
  - **Proper Markdown headings, bullet points, structured tables, and code blocks.**
  - **Consistent, structured, and professional content presentation.**

---

## **Final Output Requirements**

Generate a **fully structured and professional final report** in Markdown format with:
- **A well-structured Project Overview** that provides context and introduces key objectives.
- **A professionally refined Data & Methodologies section** that aligns with findings in 'Results & Evaluation'.
- **The most polished 'Results & Evaluation' section**, integrating the strongest aspects of the different candidate versions.
- **Properly formatted code blocks and visual elements** for readability and technical precision.

### **Final Deliverables:**
- The **full project report**, ensuring a **highly polished 'Results & Evaluation'** while maintaining professional quality across all preceding sections.
- **Clear Markdown formatting** with structured documentation, tables, figures, and code blocks.
- **Seamless logical flow** between all sections, ensuring high readability and comprehension.

---

## **Task Directive**

Your task is to **generate the full project report**, ensuring **all sections meet professional documentation standards**, with an explicit focus on refining and finalizing the 'Results & Evaluation' section to the highest level of quality.

**Produce a final, fully polished Markdown-formatted report that aligns with professional and industry standards.**


{chain_of_thought_output}

"""

# 3) Generate multiple solutions
def generate_solutions(prompt, model_name="qwen2", num_solutions=3):
    """
    Creates multiple solutions by calling the model multiple times
    with slight variations (e.g., random temperature).
    """
    solutions = []
    for i in range(num_solutions):
        # Optionally vary temperature slightly
        temp = round(random.uniform(0.6, 0.8), 2)
        payload = create_payload(
            target="open-webui",
            model=model_name,
            prompt=prompt,
            temperature=temp,
            num_ctx=150,
            num_predict=200
        )
        time_taken, response = model_req(payload=payload)
        solutions.append({
            "attempt": i + 1,
            "temperature": temp,
            "time_taken": time_taken,
            "response": response
        })
    return solutions

# 4) Self-consistency check: re-prompt the model to choose the best solution
def pick_best_solution(solutions, model_name="qwen2"):
    """
    Sends all solutions back to the model, asking it to pick the best one.
    """
    # Build a comparison prompt
    comparison_prompt = "Below are several candidate outputs for the Project Overview:\n\n"
    for s in solutions:
        comparison_prompt += f"---\nSolution {s['attempt']} (temp={s['temperature']}):\n{s['response']}\n\n"
    comparison_prompt += """
# **Professional AI-Driven Selection Prompt for Full Project Report with Emphasis on 'Results & Evaluation'**

## **Objective**

You are an advanced AI-driven technical report generator responsible for producing a **fully structured, highly professional, and insight-driven** comprehensive project report. While the **primary focus** is to meticulously refine and finalize the **'Results & Evaluation'** section, you must also ensure that all **preceding sections** (such as Project Overview, Data & Methodologies) are written at a professional standard, maintaining clarity, accuracy, and logical coherence. The entire report must align with industry standards, ensuring structured documentation, seamless readability, and technically sound content.

---

## **Step 1: AI Thought Process & Understanding**

Before selecting the best version, analyze the **purpose and key requirements** of the full report, with a primary focus on the 'Results & Evaluation' section:

- How should the **Project Overview** and **Data & Methodologies** be refined to support the analysis in the 'Results & Evaluation' section?
- What key findings must be clearly communicated in 'Results & Evaluation'?
- Which evaluation metrics best demonstrate the project's success?
- How can results be structured for clarity, impact, and ease of interpretation?
- What formatting and technical presentation enhance readability and engagement?

Use these insights to guide the **selection and refinement** process for the entire report.

---

## **Step 2: Candidate Version Generation**

Three different candidate versions of the 'Results & Evaluation' section have been generated, each varying in depth, structure, and formatting. 

Each version includes:

### **2.1 Results Summary**
- Key insights and findings from the project.
- Trends, anomalies, or expected vs. actual results.
- Alignment with project objectives and goals.

### **2.2 Evaluation Metrics**
- Explanation and justification of accuracy, precision, recall, F1-score, RMSE, etc.
- Appropriate selection of metrics based on project requirements.
- Clarity in how these metrics validate the effectiveness of the project.

### **2.3 Data Visualization & Code Blocks**
- Markdown-formatted code blocks for **result computation and metric evaluation**.
- Placeholder references for figures, tables, or charts where necessary.

#### **Example Code for Evaluating Model Performance:**
```python
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

y_true = [0, 1, 1, 0, 1, 0, 1]
y_pred = [0, 1, 0, 0, 1, 1, 1]

metrics = {
    "Accuracy": accuracy_score(y_true, y_pred),
    "Precision": precision_score(y_true, y_pred),
    "Recall": recall_score(y_true, y_pred),
    "F1 Score": f1_score(y_true, y_pred)
}

for metric, value in metrics.items():
    print(f"{metric}: {value:.2f}")
```

---

## **Step 3: AI-Based Comparative Evaluation & Selection**

Compare the **three generated versions** based on the following criteria:

### **3.1 Clarity & Readability**
- Which version **best communicates** key findings and insights?
- Does the tone maintain **technical accuracy while remaining accessible**?

### **3.2 Comprehensiveness**
- Does the version include a **thorough explanation** of results and evaluation?
- Are all necessary evaluation metrics **fully justified**?

### **3.3 Logical Flow & Structure**
- Does the version transition **smoothly** from presenting results to evaluating them?
- Are the sections organized **in a logical and professional format**?

### **3.4 Technical Accuracy & Presentation**
- Is the Markdown formatting **clear, professional, and consistent**?
- Are visual elements and code blocks **well-integrated and effective**?

#### **Final Decision:**
Select the **best** version based on the **highest overall quality** without referencing the comparison process or alternative drafts.

---

## **Step 4: Final Refinement & Output Generation**

Once the best version has been selected, apply **final refinements** to ensure that:
- The **Project Overview** and **Data & Methodologies** sections are professionally structured and logically support the 'Results & Evaluation' section.
- The 'Results & Evaluation' section is **polished, structured, and logically coherent**.
- Formatting, syntax, and transitions **meet professional documentation standards**.
- Any **necessary improvements** are made without disrupting the original intent of the section.

---

## **Final Output**

Generate the **full project report**, ensuring that all sections maintain professional consistency. The **primary focus** should be on finalizing the **'Results & Evaluation'** section to the highest standard, while also ensuring that preceding sections are professionally written and aligned with the report’s findings.

### **Key Deliverables:**
- **A single, refined** full report, including **professionally structured** Project Overview, Data & Methodologies, and **the best** 'Results & Evaluation' section.
- **Integration of the strongest aspects** from all evaluated versions.
- **Accurate, justified evaluation metrics** that align with project objectives.
- **Clear Markdown formatting** for readability and professional presentation.

---

## **Task Directive**

Your task is to **think autonomously and critically** about the best way to refine and present the **full project report**. While the main focus is on 'Results & Evaluation,' all preceding sections must also be written at a **high professional standard** to ensure consistency and seamless flow.

**Generate the final project report with a primary focus on ensuring that the 'Results & Evaluation' section meets the highest professional standards, while maintaining excellence across all preceding sections.**

"""

    payload_compare = create_payload(
        target="open-webui",
        model=model_name,
        prompt=comparison_prompt,
        temperature=0.7,
        num_ctx=250,
        num_predict=300
    )
    time_compare, response_compare = model_req(payload=payload_compare)
    return time_compare, response_compare

# 5) Generate multiple solutions
solutions = generate_solutions(BASE_PROMPT, model_name="qwen2", num_solutions=3)

# 6) Ask the model to pick the best solution
time_best, best_solution_response = pick_best_solution(solutions, model_name="qwen2")

# 7) Print out the final recommended solution
print("===== SELF-CONSISTENCY BEST SOLUTION =====")
print(best_solution_response)
if time_best:
    print(f"\nTime taken for best-solution analysis: {time_best}s")

# 8) (Optional) Log solutions and best pick
os.makedirs("data", exist_ok=True)

# Log each solution
with open("data/self_consistency_solutions.csv", "a", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    # Optionally write a header row if needed:
    # writer.writerow(["timestamp", "attempt", "temperature", "time_taken", "response"])
    for s in solutions:
        writer.writerow([
            datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            s["attempt"],
            s["temperature"],
            s["time_taken"],
            s["response"].replace("\n", "\\n")
        ])

# Log the best solution
with open("data/self_consistency_best.csv", "a", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow([
        datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "best_solution",
        time_best,
        best_solution_response.replace("\n", "\\n")
    ])


Payload: {'model': 'qwen2', 'messages': [{'role': 'user', 'content': '\nYou are an advanced AI-driven technical writing system tasked with generating a **fully structured, high-quality, and professional technical report** in Markdown format. Your primary objective is to **refine and finalize a comprehensive project report** with a particular emphasis on the **\'Results & Evaluation\'** section. \n\nWhile the **main goal** is to ensure that the \'Results & Evaluation\' section is the most insightful, technically sound, and well-structured, you must also maintain **a professional and logically coherent structure** across all preceding sections, including:\n- **Project Overview** (Concise introduction to the project)\n- **Data & Methodologies** (Thorough explanation of data collection, preprocessing, and analytical techniques)\n- **Results & Evaluation** (Detailed insights, performance metrics, visualizations, and structured technical analysis)\n\nThe **final output** must be a **refined,

: 